In [1]:
import sys
sys.path.append("../src")

import generate_general_networks as generate
import torch
import matplotlib.pyplot as plt
import json
import numpy as np
import tikzplotlib
import loading_json as load
import json

In [2]:
# Load results from optimization
f = open("../optimization_results/general_optimization/medium_complex_new.json")
results = json.load(f)
f.close()

In [3]:
# Collecting network configuration
network_file = results['network_file']
f = open("../"+network_file)
network_config = json.load(f)
f.close()

T = network_config['T']
N = network_config['N']
controls = network_config['control_points']

In [4]:
# Collecting the start and final parameters
start = results['parameters'][0]
opt = results['parameters'][-1]

In [5]:
# Create the networks
start_speeds = [[torch.tensor(start[i])] for i in range(8)]
opt_speeds = [[torch.tensor(50.0)] for i in range(8)]

start_cycle = [torch.tensor(start[8]), torch.tensor(start[9])]
opt_cycle = [torch.tensor(opt[8]), torch.tensor(opt[9])]

start_network = generate.medium_complex_network(T, N, start_speeds, controls, [start_cycle], track_grad=False)
opt_network = generate.medium_complex_network(T, N, opt_speeds, controls, [opt_cycle], track_grad=False)

In [7]:
for road in start_network.roads:
    print(road.id)

south_one_fw
south_two_fw
south_three_fw
south_four_fw
south_one_bw
south_two_bw
south_three_bw
south_four_bw
north_one_fw
north_two_fw
north_one_bw
north_two_bw
middle
mainline_1fw
mainline_2fw
mainline_3fw


In [6]:
# Do the simulations
densities, _, lengths, delays, n_stops = start_network.solve_cons_law_counting()
opt_densities, _, opt_lengths, opt_delays, opt_n_stops = opt_network.solve_cons_law_counting()

Bus bus_3 reached bus stop 0 at time 21.571428298950195, should wait for 29.992795944213867 seconds
Bus bus_1 reached bus stop 0 at time 31.857133865356445, should wait for 29.79874038696289 seconds
Bus bus_2 reached bus stop 0 at time 97.64299774169922, should wait for 29.611513137817383 seconds
Bus bus_3 reached bus stop 0 at time 17.714290618896484, should wait for 29.506494522094727 seconds
Bus bus_1 reached bus stop 0 at time 24.914295196533203, should wait for 29.84310531616211 seconds
Bus bus_2 reached bus stop 0 at time 44.97145080566406, should wait for 29.58465003967285 seconds


In [7]:
# Store densities
densities_loaded = load.convert_from_tensor(densities)
bus_lengths_loaded = load.convert_from_tensor(lengths)

end_densities_loaded = load.convert_from_tensor(opt_densities)
end_bus_lengths_loaded = load.convert_from_tensor(opt_lengths)

In [8]:
with open("../general_densities/medium_complex_start.json", 'w') as fd:
    fd.write(json.dumps([densities_loaded, bus_lengths_loaded]))

In [9]:
with open("../general_densities/medium_complex_optimal.json", 'w') as fd:
    fd.write(json.dumps([end_densities_loaded, end_bus_lengths_loaded]))

In [10]:
# Convert to the same time scale:
# Use single_lane_optimal as the goal


f = open("../general_densities/medium_complex_start.json")
data = json.load(f)
f.close()
orig_densities = data[0]
orig_lengths = data[1]

f = open("../general_densities/medium_complex_optimal.json")
data_goal = json.load(f)
f.close()
goal_densities = data_goal[0]

In [11]:
goal_times = list(goal_densities['0'].keys())
goal_times = [float(t) for t in goal_times]

old_times = list(orig_densities['0'].keys())
old_times = [float(t) for t in old_times]

In [12]:
new_lengths = {i : {} for i in orig_lengths.keys()}
new_densities = {i : {} for i in orig_densities.keys()}

prev_idx = 0
for t in goal_times:
    add_incr = 0
    for t_prev, t_next in zip(old_times[prev_idx:-1], old_times[prev_idx+1:]):
        if t_prev <= t and t_next >= t:
            prev_dist = t - t_prev
            next_dist = t_next - t
            interval_length = t_next - t_prev
            prev_weight = (interval_length - prev_dist) / interval_length
            next_weight = (interval_length - next_dist) / interval_length
            for bus_id in orig_lengths.keys(): 
                new_lengths[bus_id][str(t)] = orig_lengths[bus_id][str(t_prev)]*prev_weight + orig_lengths[bus_id][str(t_next)]*next_weight
            for road_id in orig_densities.keys():
                new_densities[road_id][str(t)] = [orig_densities[road_id][str(t_prev)][i]*prev_weight + orig_densities[road_id][str(t_next)][i]*next_weight for i in range(len(orig_densities[road_id][str(t_prev)]))]
            break
        else:
            add_incr += 1

In [13]:
# Write to file
with open("../general_densities/medium_complex_start_opt_times.json", 'w') as fd:
    fd.write(json.dumps([new_densities, new_lengths]))